In [ ]:
# Bibliotecas
!pip install google-play-scraper
!pip install pandas
!pip install matplotlib

from google_play_scraper import Sort, reviews, app
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class_names = [
    "extremamente negativo",
    "nagativo",
    "neutro",
    "positivo",
    "extremamente positivo",
]

APPS_IDS = [
    "com.einnovation.temu",
    "com.shopee.br",
    "com.mercadolibre",
    "com.amazon.mShop.android.shopping",
    "com.zzkko",
]

SORT_ORDERS = [Sort.MOST_RELEVANT]
# SORT_ORDERS = [Sort.NEWEST]
# SORT_ORDERS = [Sort.MOST_RELEVANT, Sort.NEWEST]

TOTAL_COMMENTS = 3000
TOTAL_APPS = len(APPS_IDS)
TOTAL_COMMETS_PER_APP = TOTAL_COMMENTS // TOTAL_APPS
TOTAL_COMMETS_PER_SCORE = TOTAL_COMMETS_PER_APP // 5
TOTAL_COMMETS_PER_SORT_ORDER = TOTAL_COMMETS_PER_SCORE // len(SORT_ORDERS)

## Extração dos dados

### (Opcional) Extração das informações dos aplicativos

In [ ]:
app_infos = []

# Obtem as informações dos aplicativos da lista app_ids e adiciona na lista app_infos
for app_id in APPS_IDS:
    # Obtem as informações do aplicativo
    app_info = app(app_id, lang="pt", country="br")
    # Remove os comentários
    del app_info["comments"]
    # Adiciona as informações do aplicativo na lista app_infos
    app_infos.append(app_info)

In [ ]:
# Cria um DataFrame com as informações dos aplicativos
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.head()

### Extração dos reviews

In [ ]:
app_reviews = []

# Obtem os comentários dos aplicativos da lista app_ids
# Vai obter X comentários para cada nota (1 a 5) e para cada tipo de ordenação (MOST_RELEVANT e NEWEST)
# Cuidado com MOST_RELEVANT e NEWES, pois se for os dois, o número de comentários será dobrado

print(f"Total de comentários por aplicativo: {TOTAL_COMMETS_PER_APP}")
print(f"Total de comentários por nota: {TOTAL_COMMETS_PER_SCORE}")
print(f"Total de comentários por tipo de ordenação: {TOTAL_COMMETS_PER_SORT_ORDER}")

for app_id in APPS_IDS:
    # Obtem os comentários dos aplicativos
    for score in range(1, 6):  # para cada nota de 1 a 5
        for sort_order in SORT_ORDERS:  # para cada tipo de ordenação , Sort.NEWEST
            result, _ = reviews(
                app_id=app_id,
                lang="pt",
                country="br",
                sort=sort_order,  # ordena os comentários por mais recentes
                count=TOTAL_COMMETS_PER_SORT_ORDER,  # número de comentários por nota
                filter_score_with=score,  # filtra os comentários pela nota
            )
            for review in result:
                review["app_id"] = app_id  # Identifica o aplicativo do comentário
                review["sortOrder"] = (
                    "most_relevant" if sort_order == Sort.MOST_RELEVANT else "newest"
                )
            # Adiciona os comentários na lista app_reviews
            app_reviews.extend(result)

# Cria um DataFrame com os comentários dos aplicativos
app_reviews_df = pd.DataFrame(app_reviews)

In [ ]:
app_reviews_df.head(2)

In [ ]:
app_reviews_df.shape

In [ ]:
app_reviews_df["score"].value_counts()

In [ ]:
# Contar as avaliações por score
score_counts = app_reviews_df.score.value_counts()

# Plotar as contagens por sentimento
sns.barplot(x=score_counts.index, y=score_counts.values, palette="viridis")
plt.xlabel("Score")
plt.ylabel("Número de Reviews")
plt.show()

### Tratamento dos dados

In [ ]:
# Vamos converter o conjunto de dados em sentimentos:
# extremamente negativo (0) - score = 1
# negativo (1) - score == 2
# neutral (2) - score == 3
# positive (3) - score == 4
# extremamente positivo (4) - score == 5
def get_sentiment(rating):
    rating = int(rating)
    if rating == 1:
        return 0
    elif rating == 2:
        return 1
    elif rating == 3:
        return 2
    elif rating == 4:
        return 3
    else:
        return 4

In [ ]:
app_reviews_df["sentiment"] = app_reviews_df["score"].apply(get_sentiment)
app_reviews_df.sentiment.value_counts()

In [ ]:
sentiment_counts = app_reviews_df.sentiment.value_counts()

# Plotar as contagens por sentimento
sns.barplot(x=class_names, y=sentiment_counts.values, palette="viridis")
plt.xlabel("Sentimento")
plt.ylabel("Número de Reviews")
plt.show()

In [ ]:
app_reviews_df = app_reviews_df[["content", "score"]]
app_reviews_df.head()

In [ ]:
# Salva as informações dos aplicativos em um arquivo CSV
app_reviews_df.to_csv(f"reviews_{TOTAL_COMMENTS}.csv", index=None, header=True)